# Practical Exam: Customer Purchase Prediction

RetailTech Solutions is a fast-growing international e-commerce platform operating in over 20 countries across Europe, North America, and Asia. They specialize in fashion, electronics, and home goods, with a unique business model that combines traditional retail with a marketplace for independent sellers.

The company has seen rapid growth. A key part of their success has been their data-driven approach to personalization. However, as they plan their expansion into new markets, they need to improve their ability to predict customer behavior.

Their marketing team wants to predict which customers are most likely to make a purchase based on their browsing behavior.

As an AI Engineer, you will help build this prediction system. Your work will directly impact RetailTech's growth strategy and their goal of increasing revenue.


## Data Description

| Column Name | Criteria |
|------------|----------|
| customer_id | Integer. Unique identifier for each customer. No missing values. |
| time_spent | Float. Minutes spent on website per session. Missing values should be replaced with median. |
| pages_viewed | Integer. Number of pages viewed in session. Missing values should be replaced with mean. |
| basket_value | Float. Value of items in basket. Missing values should be replaced with 0. |
| device_type | String. One of: Mobile, Desktop, Tablet. Missing values should be replaced with "Unknown". |
| customer_type | String. One of: New, Returning. Missing values should be replaced with "New". |
| purchase | Binary. Whether customer made a purchase (1) or not (0). Target variable. |

# Task 1

The marketing team has collected customer session data in `raw_customer_data.csv`, but it contains missing values and inconsistencies that need to be addressed.
Create a cleaned version of the dataframe:

- Start with the data in the file `raw_customer_data.csv`
- Your output should be a DataFrame named `clean_data`
- All column names and values should match the table below.
</br>

| Column Name | Criteria |
|------------|----------|
| customer_id | Integer. Unique identifier for each customer. No missing values. |
| time_spent | Float. Minutes spent on website per session. Missing values should be replaced with median. |
| pages_viewed | Integer. Number of pages viewed in session. Missing values should be replaced with mean. |
| basket_value | Float. Value of items in basket. Missing values should be replaced with 0. |
| device_type | String. One of: Mobile, Desktop, Tablet. Missing values should be replaced with "Unknown". |
| customer_type | String. One of: New, Returning. Missing values should be replaced with "New". |
| purchase | Binary. Whether customer made a purchase (1) or not (0). Target variable. |

In [67]:
import pandas as pd

# خواندن داده‌های خام
raw_data = pd.read_csv("raw_customer_data.csv")

# جایگزینی مقادیر گم‌شده
raw_data["time_spent"].fillna(raw_data["time_spent"].median(), inplace=True)
raw_data["pages_viewed"].fillna(round(raw_data["pages_viewed"].mean()), inplace=True)
raw_data["basket_value"].fillna(0, inplace=True)
raw_data["device_type"].fillna("Unknown", inplace=True)
raw_data["customer_type"].fillna("New", inplace=True)

# اطمینان از نوع داده‌های صحیح
raw_data["customer_id"] = raw_data["customer_id"].astype(int)
raw_data["time_spent"] = raw_data["time_spent"].astype(float)
raw_data["pages_viewed"] = raw_data["pages_viewed"].astype(int)
raw_data["basket_value"] = raw_data["basket_value"].astype(float)
raw_data["purchase"] = raw_data["purchase"].astype(int)

# ذخیره دیتاست پاک‌شده
clean_data = raw_data
print(clean_data.head())


   customer_id  time_spent  pages_viewed  ...  device_type customer_type purchase
0            1   23.097867             7  ...       Mobile     Returning        0
1            2   57.092144             3  ...       Mobile     Returning        1
2            3   44.187643            14  ...       Mobile     Returning        0
3            4   36.320851            10  ...       Mobile           New        1
4            5   10.205100            16  ...       Mobile     Returning        1

[5 rows x 7 columns]


# Task 2
The pre-cleaned dataset `model_data.csv` needs to be prepared for our neural network.
Create the model features:

- Start with the data in the file `model_data.csv`
- Scale numerical features (`time_spent`, `pages_viewed`, `basket_value`) to 0-1 range
- Apply one-hot encoding to the categorical features (`device_type`, `customer_type`)
    - The column names should have the following format: variable_name_category_name (e.g., `device_type_Desktop`)
- Your output should be a DataFrame named `model_feature_set`, with all column names from `model_data.csv` except for the columns where one-hot encoding was applied.


In [68]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# خواندن داده‌ها از فایل
raw_data = pd.read_csv("model_data.csv")

# مقیاس‌گذاری ویژگی‌های عددی
scaler = MinMaxScaler()
raw_data[["time_spent", "pages_viewed", "basket_value"]] = scaler.fit_transform(raw_data[["time_spent", "pages_viewed", "basket_value"]])

# اعمال One-Hot Encoding بر روی ویژگی‌های دسته‌ای
dummy_variables = pd.get_dummies(raw_data[["device_type", "customer_type"]], prefix=["device_type", "customer_type"])

# ترکیب داده‌های اصلی با داده‌های وان-هات اینکود شده
model_feature_set = pd.concat([raw_data.drop(columns=["device_type", "customer_type"]), dummy_variables], axis=1)

# نمایش خلاصه‌ای از داده‌های پردازش‌شده
print(model_feature_set.info())
print(model_feature_set.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   customer_id              500 non-null    int64  
 1   time_spent               500 non-null    float64
 2   pages_viewed             500 non-null    float64
 3   basket_value             500 non-null    float64
 4   purchase                 500 non-null    int64  
 5   device_type_Desktop      500 non-null    uint8  
 6   device_type_Mobile       500 non-null    uint8  
 7   device_type_Tablet       500 non-null    uint8  
 8   device_type_Unknown      500 non-null    uint8  
 9   customer_type_New        500 non-null    uint8  
 10  customer_type_Returning  500 non-null    uint8  
dtypes: float64(3), int64(2), uint8(6)
memory usage: 22.6 KB
None
   customer_id  time_spent  ...  customer_type_New  customer_type_Returning
0          501    0.664167  ...                  1 

# Task 3

Now that all preparatory work has been done, create and train a neural network that would allow the company to predict purchases.

- Using PyTorch, create a network with:
   - At least one hidden layer with 8 units
   - ReLU activation for hidden layer
   - Sigmoid activation for the output layer
- Using the prepared features in `input_model_features.csv`, train the model to predict purchases. 
- Use the validation dataset `validation_features.csv` to predict new values based on the trained model. 
- Your model should be named `purchase_model` and your output should be a DataFrame named `validation_predictions` with columns `customer_id` and `purchase`. The `purchase` column must be your predicted values.


In [69]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

# خواندن داده‌ها
train_data = pd.read_csv("input_model_features.csv")
validation_data = pd.read_csv("validation_features.csv")

# مقیاس‌گذاری ویژگی‌های عددی
scaler = MinMaxScaler()
numeric_features = ["time_spent", "pages_viewed", "basket_value"]
train_data[numeric_features] = scaler.fit_transform(train_data[numeric_features])
validation_data[numeric_features] = scaler.transform(validation_data[numeric_features])

# آماده‌سازی داده‌ها برای مدل
X_train = torch.tensor(train_data.drop(columns=["customer_id", "purchase"]).values, dtype=torch.float32)
y_train = torch.tensor(train_data["purchase"].values, dtype=torch.float32).view(-1, 1)
X_val = torch.tensor(validation_data.drop(columns=["customer_id"]).values, dtype=torch.float32)

# تعریف مدل شبکه عصبی
class PurchasePredictionModel(nn.Module):
    def __init__(self, input_dim):
        super(PurchasePredictionModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 8)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(8, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# مقداردهی مدل
input_dim = X_train.shape[1]
purchase_model = PurchasePredictionModel(input_dim)

# تنظیم تابع هزینه و بهینه‌ساز
criterion = nn.BCELoss()
optimizer = optim.Adam(purchase_model.parameters(), lr=0.001)

# آموزش مدل
epochs = 50
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = purchase_model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# پیش‌بینی روی داده‌های آموزش و اعتبارسنجی
with torch.no_grad():
    train_predictions = purchase_model(X_train).numpy()
    val_predictions = purchase_model(X_val).numpy()

# تبدیل خروجی به ۰ و ۱
train_pred_labels = (train_predictions > 0.5).astype(int)

# محاسبه دقت مدل
train_accuracy = accuracy_score(y_train.numpy(), train_pred_labels)
print(f'Training Accuracy: {train_accuracy:.4f}')

# ذخیره پیش‌بینی‌ها
validation_data["purchase"] = (val_predictions > 0.5).astype(int)
validation_predictions = validation_data[["customer_id", "purchase"]]
print(validation_predictions.head())


Epoch [10/50], Loss: 0.7031
Epoch [20/50], Loss: 0.6895
Epoch [30/50], Loss: 0.6754
Epoch [40/50], Loss: 0.6605
Epoch [50/50], Loss: 0.6448
Training Accuracy: 0.7762
   customer_id  purchase
0         1801         1
1         1802         1
2         1803         1
3         1804         1
4         1805         1
